# 63. NeRF入門 - Neural Radiance Fields

**難易度**: ★★★★★（上級）

## 学習目標

このノートブックを完了すると、以下ができるようになります：

- [ ] NeRFの基本アーキテクチャを理解する
- [ ] 位置エンコーディングの役割を説明できる
- [ ] 階層的サンプリング（Coarse-to-Fine）の仕組みを理解する
- [ ] シンプルなNeRFを実装できる
- [ ] NeRFの発展形（Instant-NGP, 3DGS等）への橋渡しを理解する

## 前提知識

- Ray Castingと座標系（Notebook 61）
- ボリュームレンダリング（Notebook 62）
- ニューラルネットワークの基礎（MLP）
- PyTorchの基本操作

## 目次

1. [NeRFとは](#1-nerfとは)
2. [NeRFのアーキテクチャ](#2-nerfのアーキテクチャ)
3. [位置エンコーディング](#3-位置エンコーディング)
4. [階層的サンプリング](#4-階層的サンプリング)
5. [実装（NumPyベース）](#5-実装numpyベース)
6. [発展的なトピック](#6-発展的なトピック)
7. [まとめとセルフチェック](#7-まとめとセルフチェック)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import Tuple, List, Optional, Callable
import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(precision=4, suppress=True)
print("ライブラリの読み込み完了")

---

## 1. NeRFとは

### 1.1 Neural Radiance Fields の概要

**NeRF (Neural Radiance Fields)** は、2020年にMildenhallらによって提案された、ニューラルネットワークを使って3Dシーンを表現・レンダリングする手法です。

**キーアイデア**: シーン全体を1つのニューラルネットワーク（MLP）で表現し、任意の位置・方向からの色と密度を出力する。

$$
F_\theta: (\mathbf{x}, \mathbf{d}) \rightarrow (\mathbf{c}, \sigma)
$$

- 入力: 3D位置 $\mathbf{x} = (x, y, z)$ と視線方向 $\mathbf{d} = (\theta, \phi)$
- 出力: RGB色 $\mathbf{c} = (r, g, b)$ と体積密度 $\sigma$

### 1.2 NeRFの革新性

| 従来手法 | NeRF |
|---------|------|
| 明示的な3D表現（メッシュ、点群） | 暗黙的な連続表現 |
| 離散的 | 任意の解像度 |
| 表面のみ | 半透明・霧も表現可能 |
| ビュー依存効果が困難 | 視線依存の色を自然に学習 |

In [ ]:
# NeRFの概念図
fig = plt.figure(figsize=(16, 6))

# 左: 従来の3D表現
ax1 = fig.add_subplot(131, projection='3d')
ax1.set_title('従来: 明示的3D表現\n（メッシュ/点群）', fontsize=11)

# 立方体のワイヤーフレーム
vertices = np.array([
    [0,0,0], [1,0,0], [1,1,0], [0,1,0],
    [0,0,1], [1,0,1], [1,1,1], [0,1,1]
])
edges = [[0,1],[1,2],[2,3],[3,0],[4,5],[5,6],[6,7],[7,4],
         [0,4],[1,5],[2,6],[3,7]]
for e in edges:
    ax1.plot3D(*zip(vertices[e[0]], vertices[e[1]]), 'b-', linewidth=2)
ax1.scatter(*vertices.T, c='blue', s=50)
ax1.set_xlabel('X'); ax1.set_ylabel('Y'); ax1.set_zlabel('Z')

# 中央: NeRFの入出力
ax2 = fig.add_subplot(132)
ax2.set_title('NeRF: ニューラル表現\nF(x,d) → (c, σ)', fontsize=11)

# MLP図
layers = [6, 8, 8, 8, 4]
x_pos = np.linspace(0.1, 0.9, len(layers))
colors_layer = ['green', 'purple', 'purple', 'purple', 'orange']
for i, (x, n, col) in enumerate(zip(x_pos, layers, colors_layer)):
    y_positions = np.linspace(0.2, 0.8, n)
    ax2.scatter([x]*n, y_positions, s=100, c=col, alpha=0.8)
    if i < len(layers) - 1:
        for y1 in y_positions:
            y_next = np.linspace(0.2, 0.8, layers[i+1])
            for y2 in y_next[:3]:  # 一部だけ描画
                ax2.plot([x, x_pos[i+1]], [y1, y2], 'gray', alpha=0.1)

ax2.text(0.05, 0.5, '(x,y,z)\n(θ,φ)', ha='right', va='center', fontsize=10, color='green')
ax2.text(0.95, 0.6, '(r,g,b)', ha='left', va='center', fontsize=10, color='orange')
ax2.text(0.95, 0.4, 'σ', ha='left', va='center', fontsize=12, color='orange')
ax2.set_xlim(0, 1); ax2.set_ylim(0, 1)
ax2.axis('off')

# 右: レンダリング結果
ax3 = fig.add_subplot(133)
ax3.set_title('出力: フォトリアルな画像', fontsize=11)

# グラデーション画像（NeRF出力のイメージ）
img = np.zeros((50, 50, 3))
for i in range(50):
    for j in range(50):
        r = np.sqrt((i-25)**2 + (j-25)**2)
        if r < 20:
            # 球のような見た目
            depth = np.sqrt(max(0, 20**2 - r**2))
            normal_z = depth / 20
            img[i, j] = [0.3 + 0.5*normal_z, 0.2 + 0.3*normal_z, 0.6 + 0.3*normal_z]
        else:
            img[i, j] = [0.9, 0.9, 0.95]  # 背景
ax3.imshow(img)
ax3.axis('off')

plt.tight_layout()
plt.show()

### 1.3 NeRFのパイプライン

```
1. レイ生成: 各ピクセルに対してカメラからレイを生成
2. サンプリング: レイに沿って点をサンプル
3. 位置エンコーディング: 座標を高次元に変換
4. MLPクエリ: 各点で(色, 密度)を取得
5. ボリュームレンダリング: 色を統合してピクセル値を計算
6. 損失計算: Ground Truthとの誤差を計算
7. 逆伝播: ネットワークを更新
```

In [ ]:
# NeRFパイプラインの詳細図
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# 1. 入力画像
ax = axes[0, 0]
ax.set_title('1. 入力: 多視点画像', fontsize=11)
for i, angle in enumerate([0, 30, 60]):
    rect = plt.Rectangle((0.1 + i*0.28, 0.3), 0.22, 0.4,
                         facecolor=f'C{i}', alpha=0.7, edgecolor='black')
    ax.add_patch(rect)
    ax.text(0.21 + i*0.28, 0.25, f'視点{i+1}', ha='center', fontsize=9)
ax.set_xlim(0, 1); ax.set_ylim(0, 1)
ax.axis('off')

# 2. レイ生成
ax = axes[0, 1]
ax.set_title('2. レイ生成', fontsize=11)
ax.scatter([0.15], [0.5], s=150, c='blue', marker='o', label='カメラ')
for angle in np.linspace(-0.25, 0.25, 5):
    ax.arrow(0.15, 0.5, 0.7, angle, head_width=0.03, head_length=0.04,
             fc='red', ec='red', alpha=0.7)
ax.text(0.5, 0.1, 'r(t) = o + td', ha='center', fontsize=10)
ax.set_xlim(0, 1); ax.set_ylim(0, 1)
ax.axis('off')

# 3. サンプリング
ax = axes[0, 2]
ax.set_title('3. 点サンプリング', fontsize=11)
x_samples = np.linspace(0.15, 0.85, 10)
ax.plot([0.1, 0.9], [0.5, 0.5], 'r-', linewidth=2)
ax.scatter(x_samples, [0.5]*10, c='green', s=60, zorder=5)
ax.text(0.5, 0.3, 't₁, t₂, ..., tₙ', ha='center', fontsize=10)
ax.set_xlim(0, 1); ax.set_ylim(0, 1)
ax.axis('off')

# 4. 位置エンコーディング + MLP
ax = axes[1, 0]
ax.set_title('4. 位置エンコーディング + MLP', fontsize=11)
ax.text(0.1, 0.7, 'x', fontsize=12, ha='center')
ax.arrow(0.15, 0.7, 0.15, 0, head_width=0.05, head_length=0.03, fc='black')
ax.text(0.42, 0.7, 'γ(x)', fontsize=10, ha='center', 
        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
ax.arrow(0.55, 0.7, 0.15, 0, head_width=0.05, head_length=0.03, fc='black')
ax.text(0.82, 0.7, 'MLP', fontsize=10, ha='center',
        bbox=dict(boxstyle='round', facecolor='purple', alpha=0.3))
ax.text(0.5, 0.35, 'γ(x) = [sin(2⁰πx), cos(2⁰πx), ...]', ha='center', fontsize=9)
ax.set_xlim(0, 1); ax.set_ylim(0, 1)
ax.axis('off')

# 5. ボリュームレンダリング
ax = axes[1, 1]
ax.set_title('5. ボリュームレンダリング', fontsize=11)
colors_vol = plt.cm.viridis(np.linspace(0.2, 0.8, 8))
x_pos = np.linspace(0.1, 0.7, 8)
for x, c in zip(x_pos, colors_vol):
    ax.add_patch(plt.Rectangle((x, 0.4), 0.07, 0.2, facecolor=c, alpha=0.7))
ax.arrow(0.75, 0.5, 0.1, 0, head_width=0.05, head_length=0.03, fc='black')
ax.add_patch(plt.Rectangle((0.87, 0.42), 0.08, 0.16, facecolor='orange', edgecolor='black'))
ax.text(0.5, 0.25, 'C = Σ Tᵢαᵢcᵢ', ha='center', fontsize=10)
ax.set_xlim(0, 1); ax.set_ylim(0, 1)
ax.axis('off')

# 6. 損失と学習
ax = axes[1, 2]
ax.set_title('6. 損失計算と学習', fontsize=11)
ax.text(0.25, 0.7, 'Ĉ', fontsize=14, ha='center')
ax.text(0.5, 0.7, '-', fontsize=20, ha='center')
ax.text(0.75, 0.7, 'C_gt', fontsize=12, ha='center')
ax.text(0.5, 0.5, '↓', fontsize=20, ha='center')
ax.text(0.5, 0.35, 'L = ||Ĉ - C_gt||²', ha='center', fontsize=11,
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
ax.set_xlim(0, 1); ax.set_ylim(0, 1)
ax.axis('off')

plt.tight_layout()
plt.show()

---

## 2. NeRFのアーキテクチャ

### 2.1 ネットワーク構造

NeRFのMLPは以下の特徴を持ちます：

1. **密度 σ は位置のみに依存**: 物体の形状は視点に依存しない
2. **色 c は位置と方向の両方に依存**: 反射・光沢などのビュー依存効果

```
入力: γ(x) (位置エンコーディング)
  ↓
8層 MLP (256 units, ReLU)
  ↓ (5層目でスキップ接続)
出力1: σ (密度) + 特徴ベクトル
  ↓ 特徴ベクトル + γ(d) (方向)
1層 MLP (128 units)
  ↓
出力2: c (RGB色)
```

In [ ]:
# NeRFアーキテクチャの詳細図
fig, ax = plt.subplots(figsize=(16, 8))

# レイヤーの描画
def draw_layer(ax, x, y, width, height, label, color='lightblue'):
    rect = plt.Rectangle((x-width/2, y-height/2), width, height,
                         facecolor=color, edgecolor='black', linewidth=1.5)
    ax.add_patch(rect)
    ax.text(x, y, label, ha='center', va='center', fontsize=9)

# 入力
ax.text(0.02, 0.7, 'x\n(3D位置)', ha='center', va='center', fontsize=10, fontweight='bold')
ax.text(0.02, 0.3, 'd\n(方向)', ha='center', va='center', fontsize=10, fontweight='bold')

# 位置エンコーディング
draw_layer(ax, 0.1, 0.7, 0.06, 0.15, 'γ(x)\n60D', 'yellow')
draw_layer(ax, 0.1, 0.3, 0.06, 0.1, 'γ(d)\n24D', 'yellow')

# MLP層
layers_x = np.linspace(0.2, 0.65, 8)
for i, x in enumerate(layers_x):
    draw_layer(ax, x, 0.7, 0.05, 0.2, f'256\nReLU', 'lightblue')
    if i == 4:  # スキップ接続
        ax.annotate('', xy=(x-0.025, 0.55), xytext=(0.1, 0.62),
                   arrowprops=dict(arrowstyle='->', color='green', lw=1.5))
        ax.text(0.22, 0.57, 'skip', fontsize=8, color='green')

# 密度出力
draw_layer(ax, 0.72, 0.8, 0.05, 0.08, 'σ', 'orange')
draw_layer(ax, 0.72, 0.65, 0.05, 0.12, '256D\n特徴', 'lightgreen')

# 方向との結合
ax.annotate('', xy=(0.78, 0.45), xytext=(0.72, 0.59),
           arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
ax.annotate('', xy=(0.78, 0.45), xytext=(0.13, 0.3),
           arrowprops=dict(arrowstyle='->', color='black', lw=1.5))

# 色出力用MLP
draw_layer(ax, 0.82, 0.45, 0.05, 0.12, '128\nReLU', 'lightblue')
draw_layer(ax, 0.9, 0.45, 0.05, 0.1, 'RGB', 'orange')

# 矢印
for i in range(len(layers_x)-1):
    ax.annotate('', xy=(layers_x[i+1]-0.025, 0.7), xytext=(layers_x[i]+0.025, 0.7),
               arrowprops=dict(arrowstyle='->', color='gray', lw=1))

ax.annotate('', xy=(0.69, 0.7), xytext=(0.67, 0.7),
           arrowprops=dict(arrowstyle='->', color='gray', lw=1))
ax.annotate('', xy=(0.87, 0.45), xytext=(0.85, 0.45),
           arrowprops=dict(arrowstyle='->', color='gray', lw=1))

# タイトル
ax.text(0.5, 0.95, 'NeRF ネットワークアーキテクチャ', ha='center', fontsize=14, fontweight='bold')

# 注釈
ax.text(0.5, 0.05, '密度σは位置xのみに依存 / 色cは位置xと方向dの両方に依存', 
        ha='center', fontsize=10, style='italic')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')

plt.tight_layout()
plt.show()

### 2.2 なぜこの構造が効果的か

1. **密度と色の分離**: 
   - 密度 σ: 物体の「存在」を表す（視点に依存しない）
   - 色 c: 見え方を表す（視点で変化する反射など）

2. **スキップ接続**: 勾配消失を防ぎ、高周波の詳細を保持

3. **深いネットワーク**: 複雑なシーンを表現するのに十分な容量

---

## 3. 位置エンコーディング

### 3.1 なぜ位置エンコーディングが必要か

MLPは本質的に **低周波バイアス（spectral bias）** を持ち、滑らかな関数しか学習できません。

位置エンコーディングにより、高周波成分を明示的に入力に含めることで、細かいディテールを学習可能にします。

### 3.2 数学的定義

位置 $p$ に対する位置エンコーディング $\gamma(p)$:

$$
\gamma(p) = \left( \sin(2^0\pi p), \cos(2^0\pi p), \sin(2^1\pi p), \cos(2^1\pi p), ..., \sin(2^{L-1}\pi p), \cos(2^{L-1}\pi p) \right)
$$

- **NeRFでの設定**: 位置 L=10 (60次元)、方向 L=4 (24次元)

In [ ]:
def positional_encoding(x: np.ndarray, L: int = 10) -> np.ndarray:
    """
    NeRFスタイルの位置エンコーディング
    
    Parameters:
    -----------
    x : np.ndarray, shape (..., D)
        入力座標
    L : int
        周波数の数（出力次元 = 入力次元 × 2 × L）
    """
    encodings = []
    for i in range(L):
        freq = 2.0 ** i * np.pi
        encodings.append(np.sin(freq * x))
        encodings.append(np.cos(freq * x))
    return np.concatenate(encodings, axis=-1)

# テスト
x_test = np.array([0.5, 0.3, 0.7])  # 3D座標
encoded = positional_encoding(x_test, L=10)

print(f"入力次元: {len(x_test)}")
print(f"L = 10 での出力次元: {len(encoded)}")
print(f"計算: 3 × 2 × 10 = {3 * 2 * 10}")

In [ ]:
# 位置エンコーディングの効果を可視化
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1D関数: 高周波のステップ関数
x = np.linspace(0, 1, 500)
y_true = np.where((x > 0.3) & (x < 0.7), 1.0, 0.0) + \
         0.3 * np.sin(20 * np.pi * x)  # 高周波成分

# 左上: 目標関数
ax = axes[0, 0]
ax.plot(x, y_true, 'b-', linewidth=2)
ax.set_title('目標関数（高周波成分を含む）')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.grid(True, alpha=0.3)

# 右上: 位置エンコーディングなしのMLP出力（概念的）
ax = axes[0, 1]
# 低周波フィルタをかけた近似
from scipy.ndimage import gaussian_filter1d
y_low_freq = gaussian_filter1d(y_true, sigma=30)
ax.plot(x, y_true, 'b-', linewidth=1, alpha=0.5, label='目標')
ax.plot(x, y_low_freq, 'r-', linewidth=2, label='PE無しMLP出力')
ax.set_title('位置エンコーディングなし\n（低周波バイアス）')
ax.set_xlabel('x')
ax.legend()
ax.grid(True, alpha=0.3)

# 左下: 位置エンコーディングの各周波数成分
ax = axes[1, 0]
L = 6
colors = plt.cm.viridis(np.linspace(0, 1, L))
for i in range(L):
    freq = 2**i * np.pi
    ax.plot(x, np.sin(freq * x), color=colors[i], 
            label=f'sin(2^{i}πx)', linewidth=1.5, alpha=0.8)
ax.set_title(f'位置エンコーディングの成分 (L={L})')
ax.set_xlabel('x')
ax.legend(loc='upper right', fontsize=8)
ax.grid(True, alpha=0.3)

# 右下: 位置エンコーディングありのMLP出力（概念的）
ax = axes[1, 1]
# 高周波も再現できる近似
y_with_pe = y_true + np.random.normal(0, 0.02, len(y_true))  # 少しノイズを加えて表現
ax.plot(x, y_true, 'b-', linewidth=1, alpha=0.5, label='目標')
ax.plot(x, y_with_pe, 'g-', linewidth=1.5, label='PE有りMLP出力')
ax.set_title('位置エンコーディングあり\n（高周波も学習可能）')
ax.set_xlabel('x')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 4. 階層的サンプリング

### 4.1 Coarse-to-Fine戦略

NeRFは2段階のサンプリングを使用します：

1. **Coarse（粗い）ネットワーク**: 
   - $N_c$ 個の均等なサンプル点
   - シーン全体の大まかな構造を把握

2. **Fine（細かい）ネットワーク**: 
   - Coarseの出力（重み）に基づいて重要な領域を特定
   - 追加の $N_f$ 個のサンプルを重要な領域に集中

### 4.2 重要度サンプリング

Coarseネットワークの出力から計算された重み $w_i = T_i \alpha_i$ を確率分布として使用：

$$
\hat{w}_i = \frac{w_i}{\sum_j w_j}
$$

この分布から追加のサンプル点を生成します。

In [ ]:
def sample_pdf(bins: np.ndarray, weights: np.ndarray, 
               n_samples: int, det: bool = False) -> np.ndarray:
    """
    重み付き分布から逆CDF法でサンプリング
    
    Parameters:
    -----------
    bins : np.ndarray, shape (N+1,)
        ビンの境界
    weights : np.ndarray, shape (N,)
        各ビンの重み
    n_samples : int
        生成するサンプル数
    det : bool
        決定的サンプリングを使用するか
    """
    weights = weights + 1e-5  # ゼロ除算防止
    pdf = weights / np.sum(weights)
    cdf = np.cumsum(pdf)
    cdf = np.concatenate([[0], cdf])
    
    if det:
        u = np.linspace(0, 1, n_samples)
    else:
        u = np.random.uniform(size=n_samples)
    
    # 逆CDF
    inds = np.searchsorted(cdf, u, side='right') - 1
    inds = np.clip(inds, 0, len(weights) - 1)
    
    below = bins[inds]
    above = bins[inds + 1]
    t = (u - cdf[inds]) / (cdf[inds + 1] - cdf[inds] + 1e-5)
    
    return below + t * (above - below)

# デモ
near, far = 2.0, 6.0
n_coarse = 64
n_fine = 128

# Coarseサンプル（均等）
t_coarse = np.linspace(near, far, n_coarse)

# 仮の重み（物体が t=3.5 付近にあると仮定）
weights_coarse = np.exp(-((t_coarse - 3.5) / 0.5)**2)

# Fineサンプル（重要度に基づく）
bins = np.linspace(near, far, n_coarse + 1)
t_fine = sample_pdf(bins, weights_coarse, n_fine)

# 結合してソート
t_all = np.sort(np.concatenate([t_coarse, t_fine]))

print(f"Coarseサンプル数: {len(t_coarse)}")
print(f"Fineサンプル数: {len(t_fine)}")
print(f"合計: {len(t_all)}")

In [ ]:
# 階層的サンプリングの可視化
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# 1. Coarseサンプルと重み
ax = axes[0]
ax.bar(t_coarse, weights_coarse, width=0.05, alpha=0.7, color='blue', label='Coarse重み')
ax.scatter(t_coarse, np.zeros_like(t_coarse) - 0.05, c='blue', s=30, marker='^', label='Coarseサンプル')
ax.axvline(3.5, color='gray', linestyle='--', alpha=0.5, label='物体位置')
ax.set_xlabel('t')
ax.set_ylabel('重み')
ax.set_title('Step 1: Coarseネットワーク → 重み計算')
ax.legend(loc='upper right')
ax.set_xlim(near - 0.2, far + 0.2)

# 2. 重みに基づくFineサンプリング
ax = axes[1]
pdf = weights_coarse / np.sum(weights_coarse)
ax.bar(t_coarse, pdf, width=0.05, alpha=0.7, color='orange', label='PDF')
ax.scatter(t_fine, np.zeros_like(t_fine) - 0.01, c='red', s=20, marker='v', 
           alpha=0.5, label='Fineサンプル')
ax.set_xlabel('t')
ax.set_ylabel('確率密度')
ax.set_title('Step 2: 重みからPDFを計算 → Fineサンプリング')
ax.legend(loc='upper right')
ax.set_xlim(near - 0.2, far + 0.2)

# 3. サンプル密度の比較
ax = axes[2]
# ヒストグラムでサンプル密度を可視化
hist_bins = np.linspace(near, far, 30)
ax.hist(t_coarse, bins=hist_bins, alpha=0.5, color='blue', label='Coarse', density=True)
ax.hist(t_fine, bins=hist_bins, alpha=0.5, color='red', label='Fine', density=True)
ax.hist(t_all, bins=hist_bins, alpha=0.3, color='green', label='結合', density=True)
ax.axvline(3.5, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('t')
ax.set_ylabel('サンプル密度')
ax.set_title('Step 3: サンプル密度の比較')
ax.legend()
ax.set_xlim(near - 0.2, far + 0.2)

plt.tight_layout()
plt.show()

print("\n観察:")
print("- Coarseサンプル: 全範囲を均等にカバー")
print("- Fineサンプル: 物体周辺（重みが高い領域）に集中")
print("- 結合: 重要な領域を高密度でサンプリング")

---

## 5. 実装（NumPyベース）

### 5.1 シンプルなNeRF実装

In [ ]:
class SimpleNeRF:
    """
    NumPyベースのシンプルなNeRF実装（学習は省略、推論のみ）
    実際のNeRFはPyTorch/JAXで実装されます。
    """
    
    def __init__(self, pos_L: int = 10, dir_L: int = 4):
        """
        Parameters:
        -----------
        pos_L : int
            位置の位置エンコーディング周波数数
        dir_L : int  
            方向の位置エンコーディング周波数数
        """
        self.pos_L = pos_L
        self.dir_L = dir_L
        
        # 入力次元
        self.pos_dim = 3 * 2 * pos_L  # 60
        self.dir_dim = 3 * 2 * dir_L  # 24
        
        # ランダムな重み（デモ用、実際は学習で獲得）
        self._init_random_weights()
    
    def _init_random_weights(self):
        """ランダムな重みを初期化（デモ用）"""
        np.random.seed(42)
        self.W1 = np.random.randn(self.pos_dim, 64) * 0.1
        self.W2 = np.random.randn(64, 64) * 0.1
        self.W_sigma = np.random.randn(64, 1) * 0.1
        self.W_feat = np.random.randn(64, 32) * 0.1
        self.W_color = np.random.randn(32 + self.dir_dim, 3) * 0.1
    
    def positional_encoding(self, x: np.ndarray, L: int) -> np.ndarray:
        """位置エンコーディング"""
        encodings = []
        for i in range(L):
            freq = 2.0 ** i * np.pi
            encodings.append(np.sin(freq * x))
            encodings.append(np.cos(freq * x))
        return np.concatenate(encodings, axis=-1)
    
    def forward(self, positions: np.ndarray, directions: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        順伝播
        
        Parameters:
        -----------
        positions : np.ndarray, shape (N, 3)
        directions : np.ndarray, shape (N, 3)
        
        Returns:
        --------
        sigmas : np.ndarray, shape (N,)
        colors : np.ndarray, shape (N, 3)
        """
        # 位置エンコーディング
        pos_enc = self.positional_encoding(positions, self.pos_L)
        dir_enc = self.positional_encoding(directions, self.dir_L)
        
        # MLP層（簡略化）
        h = np.maximum(0, pos_enc @ self.W1)  # ReLU
        h = np.maximum(0, h @ self.W2)
        
        # 密度出力
        sigma_raw = h @ self.W_sigma
        sigmas = np.maximum(0, sigma_raw.flatten())  # ReLU (正の値のみ)
        
        # 特徴ベクトル
        features = h @ self.W_feat
        
        # 方向と結合して色を出力
        combined = np.concatenate([features, dir_enc], axis=-1)
        colors_raw = combined @ self.W_color
        colors = 1 / (1 + np.exp(-colors_raw))  # Sigmoid
        
        return sigmas, colors

# インスタンス作成
nerf = SimpleNeRF(pos_L=6, dir_L=4)
print(f"位置エンコーディング次元: {nerf.pos_dim}")
print(f"方向エンコーディング次元: {nerf.dir_dim}")

In [ ]:
def volume_render(sigmas: np.ndarray, colors: np.ndarray, 
                  deltas: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    ボリュームレンダリング
    """
    alphas = 1.0 - np.exp(-sigmas * deltas)
    
    # 累積透過率
    T = np.ones(len(sigmas))
    for i in range(1, len(sigmas)):
        T[i] = T[i-1] * (1 - alphas[i-1])
    
    weights = T * alphas
    color = np.sum(weights[:, None] * colors, axis=0)
    
    return color, weights, T

def render_rays(nerf: SimpleNeRF, origins: np.ndarray, directions: np.ndarray,
                near: float, far: float, n_samples: int) -> np.ndarray:
    """
    レイをレンダリング
    
    Parameters:
    -----------
    origins : np.ndarray, shape (N_rays, 3)
    directions : np.ndarray, shape (N_rays, 3)
    
    Returns:
    --------
    colors : np.ndarray, shape (N_rays, 3)
    """
    N_rays = len(origins)
    colors = np.zeros((N_rays, 3))
    
    # サンプリング
    t_vals = np.linspace(near, far, n_samples)
    deltas = np.ones(n_samples) * (far - near) / n_samples
    
    for i in range(N_rays):
        # レイ上のサンプル点
        points = origins[i] + t_vals[:, None] * directions[i]
        dirs = np.broadcast_to(directions[i], points.shape)
        
        # NeRFクエリ
        sigmas, rgbs = nerf.forward(points, dirs)
        
        # レンダリング
        color, _, _ = volume_render(sigmas, rgbs, deltas)
        colors[i] = color
    
    return colors

# テスト: 少数のレイをレンダリング
test_origins = np.array([[0, 0, 0], [0, 0, 0]])
test_directions = np.array([[0, 0, 1], [0.1, 0, 1]])
test_directions = test_directions / np.linalg.norm(test_directions, axis=1, keepdims=True)

colors = render_rays(nerf, test_origins, test_directions, near=2.0, far=6.0, n_samples=64)
print(f"レンダリング結果: {colors}")

In [ ]:
# 小さい画像をレンダリング
def render_image_nerf(nerf: SimpleNeRF, H: int, W: int, focal: float,
                      near: float, far: float, n_samples: int) -> np.ndarray:
    """
    NeRFで画像をレンダリング
    """
    image = np.zeros((H, W, 3))
    
    for i in range(H):
        for j in range(W):
            # ピクセル座標からレイ方向
            x = (j - W/2) / focal
            y = (i - H/2) / focal
            direction = np.array([x, y, 1.0])
            direction = direction / np.linalg.norm(direction)
            
            origin = np.array([0, 0, 0])
            
            # レンダリング
            color = render_rays(nerf, origin[None], direction[None],
                               near, far, n_samples)[0]
            image[i, j] = np.clip(color, 0, 1)
    
    return image

print("NeRFでの画像レンダリング中...")
# 非常に小さい画像でデモ（計算負荷のため）
image_nerf = render_image_nerf(nerf, H=32, W=32, focal=30,
                                near=2.0, far=6.0, n_samples=32)
print("完了!")

fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(image_nerf)
ax.set_title('NeRFレンダリング結果 (ランダム重み)\n32×32, 32サンプル/レイ')
ax.axis('off')
plt.show()

print("\n注意: これはランダムな重みを使用しているため、意味のある画像にはなりません。")
print("実際のNeRFでは、多視点画像から学習した重みを使用します。")

### 5.2 NeRF学習の擬似コード

In [ ]:
print("""
==================== NeRF学習の擬似コード (PyTorch) ====================

class NeRF(nn.Module):
    def __init__(self, pos_L=10, dir_L=4, hidden_dim=256):
        super().__init__()
        self.pos_encoder = PositionalEncoder(L=pos_L)
        self.dir_encoder = PositionalEncoder(L=dir_L)
        
        # 8層MLP for density
        self.layers = nn.ModuleList([
            nn.Linear(pos_L * 6, hidden_dim),
            *[nn.Linear(hidden_dim, hidden_dim) for _ in range(7)]
        ])
        
        # 密度と特徴出力
        self.sigma_layer = nn.Linear(hidden_dim, 1)
        self.feature_layer = nn.Linear(hidden_dim, hidden_dim)
        
        # 色出力用
        self.color_layer = nn.Sequential(
            nn.Linear(hidden_dim + dir_L * 6, 128),
            nn.ReLU(),
            nn.Linear(128, 3),
            nn.Sigmoid()
        )
    
    def forward(self, pos, dir):
        pos_enc = self.pos_encoder(pos)
        dir_enc = self.dir_encoder(dir)
        
        h = pos_enc
        for i, layer in enumerate(self.layers):
            h = F.relu(layer(h))
            if i == 4:  # Skip connection
                h = torch.cat([h, pos_enc], dim=-1)
        
        sigma = F.relu(self.sigma_layer(h))
        feature = self.feature_layer(h)
        
        color = self.color_layer(torch.cat([feature, dir_enc], dim=-1))
        
        return sigma, color

# 学習ループ
for epoch in range(num_epochs):
    for batch in dataloader:
        rays_o, rays_d, target_rgb = batch
        
        # Coarseネットワーク
        t_coarse = stratified_sample(near, far, n_coarse)
        points_coarse = rays_o + t_coarse * rays_d
        sigma_c, color_c = model_coarse(points_coarse, rays_d)
        rgb_coarse, weights_c = volume_render(sigma_c, color_c, t_coarse)
        
        # Fineネットワーク（階層的サンプリング）
        t_fine = sample_pdf(t_coarse, weights_c, n_fine)
        t_all = torch.sort(torch.cat([t_coarse, t_fine]))
        points_fine = rays_o + t_all * rays_d
        sigma_f, color_f = model_fine(points_fine, rays_d)
        rgb_fine, _ = volume_render(sigma_f, color_f, t_all)
        
        # 損失
        loss = mse_loss(rgb_coarse, target_rgb) + mse_loss(rgb_fine, target_rgb)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

========================================================================
""")

---

## 6. 発展的なトピック

### 6.1 NeRFの課題と後続研究

| 課題 | 解決策 |
|-----|-------|
| 学習が遅い（数日） | Instant-NGP: ハッシュエンコーディング |
| 推論が遅い | Plenoxels, TensoRF: 明示的な格子表現 |
| 動的シーン非対応 | D-NeRF, Nerfies: 変形フィールド |
| 制限された品質 | Mip-NeRF: マルチスケール |

### 6.2 3D Gaussian Splatting (3DGS)

2023年に登場した新しい手法で、NeRFの暗黙表現に代わり、**3Dガウシアン**を明示的に使用：

- **表現**: 数百万個の3Dガウシアン（位置、共分散、色、不透明度）
- **レンダリング**: ラスタライゼーションベース（非常に高速）
- **学習**: 微分可能なスプラッティング

In [ ]:
# NeRFファミリーの進化
fig, ax = plt.subplots(figsize=(14, 8))

# タイムライン
methods = [
    (2020, 'NeRF', '暗黙表現\nMLP', 'blue'),
    (2021, 'Mip-NeRF', 'マルチスケール', 'green'),
    (2021, 'NeRF++', 'unbounded', 'green'),
    (2022, 'Instant-NGP', 'ハッシュ\nエンコーディング', 'orange'),
    (2022, 'Plenoxels', '明示的格子', 'orange'),
    (2022, 'TensoRF', 'テンソル分解', 'orange'),
    (2023, '3D Gaussian\nSplatting', 'ガウシアン\nスプラッティング', 'red'),
]

for year, name, desc, color in methods:
    x = year - 2019.5
    y = 0.5 + 0.3 * np.random.rand()
    ax.scatter([x], [y], s=300, c=color, zorder=5)
    ax.annotate(name, (x, y + 0.1), ha='center', fontsize=10, fontweight='bold')
    ax.annotate(desc, (x, y - 0.15), ha='center', fontsize=8, style='italic')

# 矢印
ax.annotate('', xy=(4, 0.55), xytext=(0.5, 0.55),
           arrowprops=dict(arrowstyle='->', color='gray', lw=2))

ax.set_xlim(0, 4.5)
ax.set_ylim(0, 1)
ax.set_xticks([0.5, 1.5, 2.5, 3.5])
ax.set_xticklabels(['2020', '2021', '2022', '2023'])
ax.set_yticks([])
ax.set_title('NeRFファミリーの進化', fontsize=14)
ax.text(4.2, 0.55, '高速化\n高品質化', ha='left', fontsize=10)

plt.tight_layout()
plt.show()

### 6.3 実用的なリソース

**公式実装・チュートリアル**:
- NeRF公式: github.com/bmild/nerf
- Nerfstudio: nerfstudio.github.io (様々なNeRF手法の統一実装)
- 3DGS: github.com/graphdeco-inria/gaussian-splatting

**学習データセット**:
- LLFF（forward-facing scenes）
- NeRF Synthetic（合成シーン）
- Mip-NeRF 360（360度シーン）

---

## 7. まとめとセルフチェック

### 学習内容の要約

| コンポーネント | 役割 |
|--------------|------|
| MLP | シーンを暗黙的に表現 F(x,d)→(c,σ) |
| 位置エンコーディング | 高周波の詳細を学習可能に |
| 階層的サンプリング | 効率的なサンプル配置 |
| ボリュームレンダリング | 微分可能な画像生成 |

### 重要な数式

$$
\text{NeRF}: F_\theta(\gamma(\mathbf{x}), \gamma(\mathbf{d})) \rightarrow (\mathbf{c}, \sigma)
$$

$$
\text{位置エンコーディング}: \gamma(p) = (\sin(2^0\pi p), \cos(2^0\pi p), ..., \sin(2^{L-1}\pi p), \cos(2^{L-1}\pi p))
$$

$$
\text{レンダリング}: C = \sum_i T_i \alpha_i \mathbf{c}_i, \quad \alpha_i = 1 - \exp(-\sigma_i \delta_i)
$$

In [ ]:
# セルフチェッククイズ
print("="*60)
print("セルフチェッククイズ")
print("="*60)

questions = [
    {
        "q": "Q1: NeRFのMLPの入力は何か？",
        "options": ["a) RGB画像", "b) 3D点群", "c) 位置(x,y,z)と視線方向(θ,φ)", "d) 深度マップ"],
        "answer": "c",
        "explanation": "NeRFは位置と視線方向を入力とし、その点での色と密度を出力します。"
    },
    {
        "q": "Q2: 位置エンコーディングを使う主な理由は？",
        "options": ["a) 次元削減", "b) ノイズ除去", "c) MLPの低周波バイアスを克服", "d) 計算高速化"],
        "answer": "c",
        "explanation": "MLPは本質的に低周波関数しか学習できないため、位置エンコーディングで高周波成分を明示的に与えます。"
    },
    {
        "q": "Q3: 密度σと色cの入力依存性の違いは？",
        "options": ["a) 両方とも位置のみ", "b) 両方とも方向のみ", 
                   "c) σは位置のみ、cは位置と方向", "d) σは方向のみ、cは位置のみ"],
        "answer": "c",
        "explanation": "密度（物体の存在）は視点に依存しませんが、色（見え方）は反射などにより視線方向で変化します。"
    },
    {
        "q": "Q4: 階層的サンプリングでFineサンプルはどこに集中するか？",
        "options": ["a) レイの始点付近", "b) レイの終点付近", 
                   "c) Coarseの重みが高い領域", "d) ランダムな位置"],
        "answer": "c",
        "explanation": "Coarseネットワークの出力（重み）に基づいて、物体がありそうな領域に集中的にサンプルします。"
    },
    {
        "q": "Q5: 3D Gaussian Splattingの主な利点は？",
        "options": ["a) より高い画質", "b) リアルタイムレンダリングが可能", 
                   "c) 学習データが少なくて済む", "d) 動的シーンのサポート"],
        "answer": "b",
        "explanation": "3DGSはラスタライゼーションベースのレンダリングを使用するため、NeRFよりも大幅に高速（リアルタイム）です。"
    }
]

for q in questions:
    print(f"\n{q['q']}")
    for opt in q['options']:
        print(f"  {opt}")

print("\n" + "="*60)
print("解答は下のセルを実行してください")
print("="*60)

In [ ]:
# 解答表示
print("解答と解説:")
print("="*60)
for i, q in enumerate(questions):
    print(f"\nQ{i+1}: 正解は {q['answer']}")
    print(f"   {q['explanation']}")

### Unit 0.3 完了

このノートブックで、3Dコンピュータビジョンの基礎からNeRFまでの学習が完了しました！

**学習した内容（Unit 0.3 全体）**:

1. **Phase 1 (50-52)**: 光学と画像形成
   - 光学の基礎
   - ピンホールカメラモデル
   - レンズ歪みと補正

2. **Phase 2 (53-54)**: 座標系と変換
   - 3D座標変換と剛体運動
   - カメラキャリブレーション

3. **Phase 3 (55-57)**: マルチビュー幾何
   - エピポーラ幾何
   - ステレオ視と視差
   - 三角測量と3D復元

4. **Phase 4 (58-60)**: 3D再構成
   - 特徴点検出とマッチング
   - SfMパイプライン
   - バンドル調整

5. **Phase 5 (61-63)**: ニューラルレンダリングへの橋渡し
   - Ray Castingと座標系
   - ボリュームレンダリング
   - **NeRF入門** ← 現在のノートブック

---

**次のステップ**:
- 実際のNeRF実装（nerfstudio等）を試す
- 3D Gaussian Splattingを学ぶ
- 4Dビジョン（動的シーン）に進む

---

**ナビゲーション**

[← 62. ボリュームレンダリング](./62_volume_rendering_v1.ipynb) | [Unit 0.3 完了!]